In [38]:
from construct import Struct, Int16ul, Bytes, this, GreedyRange

# Ensure element and channel_header are defined before event

channel_header = Struct(
    "evt_pattern" / Int16ul,
    "evt_info" / Int16ul,
    "num_trace_blks" / Int16ul,
    "num_trace_blks_prev" / Int16ul,
    "trig_time_lo" / Int16ul,
    "trig_time_mi" / Int16ul,
    "trig_time_hi" / Int16ul,
    "trig_time_x" / Int16ul,
    "energy" / Int16ul,
    "chan_no" / Int16ul,
    "user_psa_value" / Int16ul,
    "xia_psa_value" / Int16ul,
    "extended_psa_values" / Bytes(8),
    "reserved" / Bytes(32),
)

element = Struct(
    "header" / channel_header,
    "data" / Bytes(lambda ctx: ctx.header.num_trace_blks * 2 * ctx._.header.blk_size)
)

event = Struct(
    "elements" / GreedyRange(element)
)

pixie4e_header = Struct(
    "blk_size" / Int16ul,
    "mod_num" / Int16ul,
    "run_format" / Int16ul,
    "chan_head_len" / Int16ul,
    "coinc_pat" / Int16ul,
    "coinc_win" / Int16ul,
    "max_comb_event_len" / Int16ul,
    "board_version" / Int16ul,
    "event_length_0" / Int16ul,
    "event_length_1" / Int16ul,
    "event_length_2" / Int16ul,
    "event_length_3" / Int16ul,
    "serial_number" / Int16ul,
    "reserved" / Int16ul,
)

pixie_eor = Struct(
    "evt_pattern" / Int16ul,
    "evt_info" / Int16ul,
    "num_trace_blks" / Int16ul,
    "num_trace_blks_prev" / Int16ul,
    "reserved" / Int16ul,
)

In [41]:
def pull_events(input_path, output_path, num_events):
    with open(input_path, 'rb') as f:
        header_data = f.read(64)
        header = pixie4e_header.parse(header_data)
        blk_size = header.blk_size
        chan_head_len = header.chan_head_len
        
        events_data = []

        for i in range(num_events):
            try:
                # Read data
                element_header_data = f.read(64)
                element_header = channel_header.parse(element_header_data)

                num_trace_blks = element_header.num_trace_blks
                data_size = 2 * blk_size * num_trace_blks
                element_data = f.read(data_size)
                if len(element_data) != data_size:
                    print(f"Event {i+1} element data length mismatch, expected {data_size} but got {len(element_data)}")
                    break
                events_data.append(element_header_data + element_data)
                    
            except Exception as e:
                print(f"Error parsing event {i+1}: {e}")
                break

        # Read the footer
        footer_data = f.read(64)
        footer = pixie_eor.parse(footer_data)

    # Write the new file
    with open(output_path, 'wb') as f:
        f.write(header_data)
        for event_data in events_data:
            f.write(event_data)
        f.write(footer_data)
                


# Change output_path and/or num_events to generate a new file from the first num_events from the original file
input_path = "../data/LaBr_Na22coinc_attn8_500fast_0148/LaBr_Na22coinc_attn8_500fast_0148.b00"
output_path = "../data/LaBr_Na22coinc_attn8_500fast_0148/LaBr_Na22coinc_attn8_500fast_0148_xxsmall.b00"
num_events = 3

pull_events(input_path, output_path, num_events)